In [41]:
%reset

In [42]:
import matplotlib.pyplot as plt
import numpy as np
import scipy.io
import pandas as pd
import pickle
import copy
import pandas as pd
from tqdm.notebook import tqdm_notebook
import os

In [43]:
mat_dir = 'C:\\Users\\User\\Cloud-Drive\\BigFiles\\OmissionExpOutput\\\\imported_eventDetectionChan\\filter0.35'
prepro_name = "imported"
import_type = "eventDetectionChan"
dir_scoring = "C:\\Users\\User\\Cloud-Drive\\BigFiles\\AnatArzData\\Data\\FinalScores"

In [44]:
all_subjects = np.array(
    ['08', '09', '10','11', '12','13', '14', '15', '16','17', '19', '20', '21', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34','35', '36', '37', '38'])
outlier_subs_major_exp_fault = ['12','18','22']
valid_subs = np.delete(all_subjects,np.isin(all_subjects,outlier_subs_major_exp_fault))
subs = valid_subs

cfg = {}
cfg['outputs_dir_path'] =  'C:\\Users\\User\\Cloud-Drive\\BigFiles\\OmissionExpOutput'
cfg['mat_files_dir'] =  mat_dir

cfg['prepro_name'] = prepro_name
cfg['electrodes_names'] = np.asarray(['E6','E7','E13','E106','E112'])
cfg['subs'] = valid_subs
cfg['outlier_subs'] = outlier_subs_major_exp_fault

In [45]:
all_files_dic = {}
for sub in subs:
    for sleep_i in np.arange(1,6):
        sub_load_filepath = f"{cfg['mat_files_dir']}\sub{sub}_sleep{sleep_i}_{prepro_name}_{import_type}.mat"
        try: 
            matlabImport = scipy.io.loadmat(sub_load_filepath, simplify_cells=True)
            all_files_dic[f'{sub}_{sleep_i}'] = {}
            all_files_dic[f'{sub}_{sleep_i}']['events'] = pd.DataFrame(matlabImport['subEvents'])
            all_files_dic[f'{sub}_{sleep_i}']['data'] = matlabImport[f'subData']
            all_files_dic[f'{sub}_{sleep_i}']['subnum'] = sub
            all_files_dic[f'{sub}_{sleep_i}']['filenum'] = sleep_i

            datafile_scoring = f"{dir_scoring}\s_{sub}_{sleep_i}"
            readEx = pd.read_excel(f'{datafile_scoring}.xlsx',header=None) # header =None is important, otherwise the scoring will be shifted
            all_files_dic[f'{sub}_{sleep_i}']['scoring'] = [x[0] for x in readEx.to_numpy()]

        except Exception: continue #print(f'Error importing sub file at: {sub_load_filepath}')


cfg['sample_freq'] = matlabImport[f'samplefreq']
cfg['electrodes_names'] = matlabImport[f'chansNames']

In [50]:
len(all_files_dic[f'32_2']['scoring'][0:20])
all_files_dic[f'32_2']['scoring'][96]

5

## Save

In [47]:
with open(f'{mat_dir}\\{import_type}.pkl', 'wb') as file:
    pickle.dump([all_files_dic, cfg], file)

# Load
# with open(f'{epochs_dir}\\{import_type}_allEvents_df.pkl', 'rb') as file:
#     _allEvents_df = pickle.load(file)
# with open(f'{epochs_dir}\\{import_type}_allEpochs.pkl', 'rb') as file:
#     _allEpochs_perCond = pickle.load(file)